In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from prophet import Prophet

# Charger les deux feuilles du fichier Excel
file_path = "../data/online_retail_II.xlsx"
df1 = pd.read_excel(file_path, sheet_name="Year 2009-2010")
df2 = pd.read_excel(file_path, sheet_name="Year 2010-2011")

# Combiner les datasets
df = pd.concat([df1, df2], ignore_index=True)

# Nettoyage des données
df.dropna(subset=["Customer ID", "Description"], inplace=True)
df = df[df["Quantity"] > 0]
df = df[df["Price"] > 0]

# Création de colonnes utiles
df["TotalPrice"] = df["Quantity"] * df["Price"]
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
df["Date"] = df["InvoiceDate"].dt.date

# ➤ Visualisation : Ventes journalières (matplotlib/seaborn)
daily_sales = df.groupby("Date")["TotalPrice"].sum().reset_index()

plt.figure(figsize=(12, 5))
sns.lineplot(data=daily_sales, x="Date", y="TotalPrice")
plt.title("Évolution des ventes journalières")
plt.xlabel("Date")
plt.ylabel("Chiffre d'affaires (TotalPrice)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# ➤ Visualisation interactive : Ventes journalières (Plotly)
fig1 = px.line(daily_sales, x="Date", y="TotalPrice", title="Ventes journalières (Plotly)")
fig1.show()

# ➤ Visualisation des produits les plus vendus
top_products = df.groupby("Description")["TotalPrice"].sum().nlargest(10).reset_index()
fig2 = px.bar(top_products, x="TotalPrice", y="Description", orientation='h',
              title="Top 10 Produits par chiffre d'affaires")
fig2.show()

# ➤ Visualisation des meilleurs pays (hors UK)
top_countries = df[df["Country"] != "United Kingdom"].groupby("Country")["TotalPrice"].sum().nlargest(10).reset_index()
fig3 = px.bar(top_countries, x="TotalPrice", y="Country", orientation='h',
              title="Top 10 Pays par chiffre d'affaires (hors UK)")
fig3.show()

# ➤ Préparation des données pour Prophet
daily_sales_prophet = daily_sales.rename(columns={"Date": "ds", "TotalPrice": "y"})

# ➤ Prévision avec Prophet
model = Prophet()
model.fit(daily_sales_prophet)
future = model.make_future_dataframe(periods=30)
forecast = model.predict(future)

# ➤ Affichage des prévisions
fig4 = px.line(forecast, x="ds", y=["yhat", "yhat_lower", "yhat_upper"],
               title="Prévision des ventes (Prophet)")
fig4.show()


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.